Embedding 模型生成向量的一般流程
Embedding 模型的核心目标是将离散的文本数据转换为稠密的、低维的、能够捕捉语义信息的向量。这个过程通常涉及以下几个关键步骤：

文本预处理与词元化 (Tokenization)：

文本清洗：去除不必要的字符、转换大小写（视模型而定）、处理特殊符号等。
词元化：将文本分割成更小的单元，称为“词元”(tokens)。现代模型（如 Transformer 类模型）通常使用子词切分算法，如 BPE (Byte Pair Encoding)、WordPiece 或 SentencePiece。这样做的好处是能够处理未登录词 (Out-of-Vocabulary, OOV)，并能更好地捕捉词缀等形态信息。例如，“transformer” 可能会被切分为 "transform" 和 "##er"。
添加特殊词元：根据模型的需要，在词元序列的开头和/或结尾添加特殊词元。例如，BERT 及其变体通常会在序列开头添加 [CLS] (Classifier) 词元，在序列间或序列末尾添加 [SEP] (Separator) 词元。对于 bge-m3 这类句子嵌入模型，[CLS] 词元通常用于聚合整个句子的信息。
转换为输入ID：将每个词元（包括特殊词元）映射到其在模型词汇表中的唯一整数 ID。
生成注意力掩码 (Attention Mask)：创建一个与输入ID序列等长的二进制掩码，用于指示哪些词元是真实的词元（值为1），哪些是填充词元（值为0，如果进行了填充操作）。这确保模型在进行自注意力计算时只关注真实的词元。
(可选) 生成词元类型ID (Token Type IDs)：对于需要处理句子对的任务（如判断两个句子是否是下一句关系），会为每个词元分配一个类型ID，以区分它属于第一个句子还是第二个句子。对于单句嵌入任务，通常所有词元都属于同一个类型。
嵌入层 (Embedding Layer)：

词嵌入 (Token Embeddings)：将每个词元的整数 ID 转换为一个稠密的向量。这通常是通过一个大的查找表（嵌入矩阵）实现的，该矩阵在模型预训练阶段学习得到。
位置嵌入 (Positional Embeddings)：由于 Transformer 模型本身不包含序列顺序的信息（其自注意力机制是置换不变的），因此需要额外引入位置信息。位置嵌入会为序列中的每个位置生成一个向量，并将其加到对应的词嵌入上。
词元类型嵌入 (Token Type Embeddings)：如果使用了词元类型ID，也会将这些ID转换为向量并加到总的嵌入表示上。
最终，每个词元的初始表示是其词嵌入、位置嵌入和（可能的）词元类型嵌入的总和。
编码器 (Encoder)：

这是模型的核心部分，通常由多层相同的 Transformer 编码器层堆叠而成。bge-m3 就是一个基于 Transformer 编码器架构的模型。
每个编码器层主要包含两个子模块：
多头自注意力机制 (Multi-Head Self-Attention)：允许模型在处理每个词元时，同时关注输入序列中的所有其他词元，并根据相关性计算加权和，从而动态地捕捉上下文信息。多头机制则允许模型从不同表示子空间学习不同方面的依赖关系。
前馈神经网络 (Feed-Forward Network)：通常是一个两层的全连接网络，独立地应用于每个位置的表示。
每个子模块之后通常会接一个残差连接 (Residual Connection) 和层归一化 (Layer Normalization)，以帮助稳定训练和加速收敛。
输入序列的嵌入表示经过编码器多层的处理后，输出的是每个词元在深层上下文中的表示向量。此时，我们得到的是一个与输入词元序列等长的向量序列。
池化层 (Pooling Layer)：

编码器的输出是每个词元的高维表示。为了获得整个输入文本（例如一个句子）的单一固定大小的向量表示，需要进行池化操作。常见的池化策略有：
CLS 池化 (CLS Pooling)：直接取用 [CLS] 特殊词元对应的输出向量作为整个序列的表示。这种策略在 BERT 及其变体中非常流行，因为 [CLS] 词元在预训练过程中就被设计用来聚合整个序列的信息。bge-m3 推荐使用此策略。
平均池化 (Mean Pooling)：将所有（或非填充）词元的输出向量进行平均（通常是元素级别的平均）。有时会考虑注意力掩码，只对真实词元进行平均。
最大池化 (Max Pooling)：在所有词元输出向量的每个维度上取最大值。
（可选）归一化 (Normalization)：

许多先进的嵌入模型（包括 bge-m3）会在池化后的向量上应用 L2 归一化，使得生成的向量长度为1。这样做的好处是，在使用余弦相似度比较向量时，点积运算就等价于余弦相似度，并且可以使向量分布在单位超球面上，有助于改善检索等任务的性能。
使用 transformers 加载本地 bge-m3 模型生成向量（底层演示）
我们将不直接使用 sentence-transformers 库（它对这个过程做了很多封装），而是使用 transformers 库的 AutoModel 和 AutoTokenizer 来手动执行上述步骤。``

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# 1. 指定本地模型路径
model_path = r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3" # 使用r""避免转义问题


c:\Users\k\.conda\envs\bge\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# 2. 加载分词器 (Tokenizer) 和模型 (Model)
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
except OSError as e:
    print(f"加载模型或分词器失败，请检查路径是否正确或模型文件是否完整: {e}")
    print(f"尝试加载的路径是: {model_path}")
    exit()

# 确保模型处于评估模式
model.eval()


XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elem

In [3]:

# 检查是否有可用的GPU，并据此设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"使用的设备: {device}")


使用的设备: cpu


In [4]:

# 3. 准备输入文本
sentences = [
    "什么是大型语言模型？",
    "bge-m3 是一个强大的文本嵌入模型。",
    "我喜欢用Python进行编程。"
]



4. 文本预处理与词元化 (Tokenization)
   - padding=True: 将批次内的句子填充到相同长度
   - truncation=True: 如果句子超过max_length，则截断
   - return_tensors="pt": 返回PyTorch张量
   - max_length: bge-m3 可以处理较长序列，但这里我们设置一个常规值。
     对于bge-m3，其上下文长度支持到8192，但一般应用不需要那么长。
     Instruction中通常会建议使用 `[CLS]` 和 `[SEP]` (如果需要，但bge-m3通常在单句时，tokenizer会自动处理)
     bge-m3的官方用法建议在文本前添加指令（用于检索任务），但这里我们仅做通用嵌入。
     对于通用句子嵌入，通常不需要特定指令。


In [5]:

encoded_input = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=512, # 你可以根据bge-m3的具体建议调整
    return_tensors='pt'
)

# 将输入数据移动到与模型相同的设备
encoded_input = {key: val.to(device) for key, val in encoded_input.items()}
# encoded_input 包含: 'input_ids', 'token_type_ids' (对于单句通常全0), 'attention_mask'


In [6]:

print("\n词元化后的输入 (input_ids):")
print(encoded_input['input_ids'])
print("形状:", encoded_input['input_ids'].shape) # (batch_size, sequence_length)

print("\n注意力掩码 (attention_mask):")
print(encoded_input['attention_mask'])
print("形状:", encoded_input['attention_mask'].shape) # (batch_size, sequence_length)



词元化后的输入 (input_ids):
tensor([[     0,      6, 171815,  23032,  38046,  71277,     32,      2,      1,
              1,      1,      1,      1,      1,      1],
        [     0,    876,    429,      9,     39,    363,      6,  13663, 101863,
         189061, 226614,   2283,  71277,     30,      2],
        [     0,  13129,  14999,   1173,    683,     53,  50828,   3327, 225691,
             30,      2,      1,      1,      1,      1]])
形状: torch.Size([3, 15])

注意力掩码 (attention_mask):
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])
形状: torch.Size([3, 15])


In [7]:

# 5. 通过模型获取所有词元的隐藏状态 (Encoder output)
#    我们不需要计算梯度，所以使用 torch.no_grad() 来节省内存和计算
with torch.no_grad():
    # 将编码后的输入传递给模型
    # **encoded_input 会将字典解包为 input_ids=..., attention_mask=... 等参数
    model_output = model(**encoded_input)
    # model_output 是一个 BaseModelOutputWithPooling 对象 (或类似对象)
    # model_output.last_hidden_state 包含了所有词元的最后一层隐藏状态
    # 形状: (batch_size, sequence_length, hidden_size)
    # hidden_size 对于 bge-m3 来说是 1024
    all_token_embeddings = model_output.last_hidden_state

print(f"\n所有词元的最后一层隐藏状态形状: {all_token_embeddings.shape}")



所有词元的最后一层隐藏状态形状: torch.Size([3, 15, 1024])


In [10]:

# 6. 池化操作 (Pooling) - 对于bge-m3，推荐使用CLS池化
#    [CLS] 词元通常是序列中的第一个词元 (index 0)
#    它的ID通常是 tokenizer.cls_token_id
#    例如，对于bge-m3的分词器，tokenizer.cls_token_id 应该是101 (如果是BERT风格的tokenizer)
#    或者更安全的是，检查一下，通常它就是第一个词元。
#    bge-m3 的文档通常指明使用第一个 token ([CLS]) 的输出。
sentence_embeddings_before_norm = all_token_embeddings[:, 0]
# 形状: (batch_size, hidden_size)
sentence_embeddings_before_norm.shape

torch.Size([3, 1024])

In [11]:

print(f"\nCLS池化后的句子向量形状 (归一化前): {sentence_embeddings_before_norm.shape}")

# 7. 归一化 (Normalization) - bge-m3 推荐使用L2归一化
#    torch.nn.functional.normalize(tensor, p=2, dim=dimension_to_normalize)
final_embeddings = F.normalize(sentence_embeddings_before_norm, p=2, dim=1)
# 形状: (batch_size, hidden_size)

print(f"\nL2归一化后的最终句子向量形状: {final_embeddings.shape}")

print("\n生成的向量 (前两个句子的前5个维度):")
for i in range(min(2, len(sentences))): # 最多打印前两个句子的向量
    print(f"句子 '{sentences[i]}': {final_embeddings[i, :5].cpu().numpy()}...") # .cpu().numpy() 用于打印

print("\n向量示例 (第一个句子的完整向量的前10个维度):")
print(final_embeddings[0, :10].cpu().numpy())


CLS池化后的句子向量形状 (归一化前): torch.Size([3, 1024])

L2归一化后的最终句子向量形状: torch.Size([3, 1024])

生成的向量 (前两个句子的前5个维度):
句子 '什么是大型语言模型？': [-0.03327692 -0.03054914 -0.00830118 -0.02365865 -0.00107772]...
句子 'bge-m3 是一个强大的文本嵌入模型。': [-0.04163737 -0.02466566 -0.01237853 -0.02900485  0.01531815]...

向量示例 (第一个句子的完整向量的前10个维度):
[-0.03327692 -0.03054914 -0.00830118 -0.02365865 -0.00107772 -0.00756135
  0.00985107 -0.01994906 -0.00338586 -0.04969517]
